In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-30"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16401,2024-03-30,Espanha Acb,14:00,Breogan,Bilbao,1.61,2.33,157.5,1.86,1.94,-4.5,2.04,3.10,zoTOjLAg,0.621118,0.429185,0.537634,0.515464,0.050303,0.8,0.8,NaN,NaN,276.336,134.830952,0.487924,211.974,45.444214,0.214386,208.700,334.798,448.95,184.07,1.0,0.0,0.0,0.0,0.258435,0.029773,0.291647,-1.99,-0.398,-1.532663,0.000000,0.0,0.000000,-5.00,-1.000,-1.330000,0.000000,0.0,0.000000
16402,2024-03-30,Eua Nba,18:00,New Orleans Pelicans,Boston Celtics,2.71,1.50,222.5,1.85,1.93,4.5,1.99,1.27,lpDLNiNK,0.369004,0.666667,0.540541,0.518135,0.035670,0.4,0.6,NaN,NaN,204.162,19.852136,0.097237,166.534,20.219124,0.121411,191.182,174.694,222.56,200.56,0.0,0.0,0.0,0.0,0.406460,0.029930,0.312342,-0.30,-0.060,-28.500000,0.667771,0.6,-0.067771,0.39,0.078,6.410256,0.669731,0.8,0.130269
16403,2024-03-30,Itália Liga A,15:00,Basket Napoli,Venezia,1.88,1.92,164.5,1.80,1.99,-1.5,2.00,2.05,fwzLSQFc,0.531915,0.520833,0.555556,0.502513,0.052748,0.6,0.6,NaN,NaN,253.706,125.507051,0.494695,138.740,33.865481,0.244093,134.656,134.872,193.39,142.20,0.0,0.0,0.0,0.0,0.014886,0.070897,0.017459,-0.90,-0.180,-4.888889,0.000000,0.0,0.000000,-1.79,-0.358,-2.569832,0.555190,0.4,-0.155190
16404,2024-03-30,Itália Liga A,16:00,Brindisi,Sassari,1.80,2.02,158.5,1.86,1.94,-2.5,2.05,2.26,GdvV538p,0.555556,0.495050,0.537634,0.515464,0.050605,0.6,0.2,NaN,NaN,260.850,209.995800,0.805044,186.120,79.518210,0.427242,185.452,187.738,157.53,163.40,0.0,0.0,0.0,0.0,0.081447,0.029773,0.068906,1.83,0.366,2.185792,0.470155,0.5,0.029845,-3.33,-0.666,-1.531532,0.378006,0.1,-0.278006
16405,2024-03-30,Itália Liga A,17:00,Varese,Brescia,2.94,1.41,170.5,1.88,1.88,4.5,2.05,1.20,EscY4NNj,0.340136,0.709220,0.531915,0.531915,0.049356,0.4,0.4,NaN,NaN,205.344,46.964285,0.228710,140.574,26.020878,0.185104,218.764,142.110,122.85,167.44,0.0,0.0,0.0,0.0,0.497413,0.000000,0.369871,-1.43,-0.286,-6.783217,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
16406,2024-03-30,Alemanha Bbl,14:30,Heidelberg,Crailsheim Merlins,1.48,2.68,177.5,1.80,1.94,-4.5,1.86,3.70,WzZDxihn,0.675676,0.373134,0.555556,0.515464,0.048810,0.6,0.4,NaN,NaN,338.024,195.029306,0.576969,526.602,571.249152,1.084783,212.882,622.620,153.12,339.30,0.0,0.0,0.0,0.0,0.407946,0.052938,0.468013,-3.26,-0.652,-0.736196,0.506084,0.1,-0.406084,-2.14,-0.428,-3.925234,0.000000,0.0,0.000000
16407,2024-03-30,Alemanha Bbl,16:00,Bonn,Vechta,1.63,2.29,171.5,1.85,1.93,-2.5,1.82,2.64,UeEsW6yU,0.613497,0.436681,0.540541,0.518135,0.050178,0.6,0.6,NaN,NaN,149.100,32.609614,0.218710,149.592,30.954717,0.206928,146.826,195.704,156.80,150.88,0.0,0.0,0.0,0.0,0.238107,0.029930,0.260012,-0.91,-0.182,-3.461538,0.000000,0.0,0.000000,2.83,0.566,2.279152,0.540746,0.7,0.159254
16408,2024-03-30,Argentina Liga A,11:00,San Martin,Quimsa,2.37,1.54,156.5,1.87,1.79,3.5,1.91,1.31,jL7d5OvP,0.421941,0.649351,0.534759,0.558659,0.071292,0.6,0.4,NaN,NaN,123.014,32.145056,0.261312,119.820,29.653580,0.247484,134.474,134.720,94.38,171.68,0.0,0.0,0.0,0.0,0.300204,0.030912,0.263518,1.95,0.390,3.512821,0.000000,0.0,0.000000,3.56,0.712,0.758427,0.000000,0.0,0.000000
16409,2024-03-30,Argentina Liga A,21:15,Obras,Instituto de Cordoba,1.98,1.75,161.5,1.85,1.85,1.5,1.85,1.61,8b1exvUs,0.505051,0.571429,0.540541,0.540541,0.076479,0.4,0.8,NaN,NaN,159.746,51.439334,0.322007,104.870,26.466802,0.252377,117.488,118.990,127.98,139.37,0.0,0.0,0.0,0.0,0.087204,0.000000,0.098096,-2.47,-0.494,-1.983806,0.000000,0.0,0.000000,0.37,0.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Espanha Acb,Breogan,Bilbao,157.5,1.86,1.0000,Over
1,15:00,Itália Liga A,Basket Napoli,Venezia,164.5,1.80,1.0000,Over
2,16:00,Itália Liga A,Brindisi,Sassari,158.5,1.86,1.0000,Over
3,14:30,Alemanha Bbl,Heidelberg,Crailsheim Merlins,177.5,1.80,1.0000,Over
4,16:00,Alemanha Bbl,Bonn,Vechta,171.5,1.85,1.0000,Over
5,21:49,Eua Ncaa,Alabama,Clemson,164.5,1.91,1.0000,Over
6,17:00,Europa Liga Aba,Cibona,Igokea,158.5,1.85,1.0000,Over
7,14:30,França Lnb,Ada Blois,Le Mans,164.5,1.84,1.0000,Over
8,14:30,França Lnb,Nancy,Cholet,163.5,1.86,1.0000,Over
9,14:00,Hungria Nacional,Budapesti Honved Se,Zalakeramia,163.5,1.82,1.0000,Over
